In [1]:
!ls -a

'ls' は、内部コマンドまたは外部コマンド、
操作可能なプログラムまたはバッチ ファイルとして認識されていません。


In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

import utils.read_data as rd
import utils.io_model as io_m
import utils.preprocessing as pp
import utils.prepare_data as prepare_data

In [3]:
import os
from os.path import join, dirname
from dotenv import load_dotenv
from pathlib import Path

In [4]:
load_dotenv(verbose=True)
dotenv_path = join(Path().resolve(), '.env')
load_dotenv(dotenv_path)

True

In [5]:
GOOGLE_DRIVE_PATH = os.environ.get("GOOGLE_DRIVE_PATH") + '/horse_racing'
DATA_PATH = GOOGLE_DRIVE_PATH + '/csv/train_data'

In [6]:
print(GOOGLE_DRIVE_PATH)

C:\Users\endo_\Dropbox\workspace\HorseRacing/horse_racing


In [7]:
df = rd.read_horse_race_csv(DATA_PATH)

horse_race_20201128.csv


c:\users\endo_\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (17,65,113,157,161,205,209,253,257) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639818 entries, 0 to 639817
Columns: 288 entries, race_id to rentan3-5
dtypes: float64(89), int64(13), object(186)
memory usage: 1.4+ GB


In [9]:
df.head()

,race_id,rank,frame_number,horse_number,horse_id,sex_and_age,burden_weight,rider_id,goal_time,goal_time_dif,...,hukusyo_second-5,hukusyo_third-5,wakuren-5,umaren-5,wide_1_2-5,wide_1_3-5,wide_2_3-5,umatan-5,renhuku3-5,rentan3-5
0,2.017091e+11,11,5,8,2015100928,牝2,51.0,1168,1:11.2,1.1/4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.017091e+11,9,6,11,2015100928,牝2,54.0,1126,1:14.9,アタマ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.018080e+11,12,2,3,2015100928,牝3,54.0,732,1:30.8,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.018080e+11,4,2,4,2015100928,牝3,54.0,1154,1:14.4,1.3/4,...,260,190,"6,240","17,470","4,710","2,280","1,030","37,440","22,660","303,790"
4,2.018090e+11,16,6,12,2015100928,牝3,54.0,732,1:14.1,クビ,...,170,210,"2,680","2,560",740,"1,170",500,"6,700","3,970","32,740"


## 前処理

In [10]:
df_for_learning = prepare_data.prepare_train_data(df)

C:\Users\endo_\Dropbox\workspace\HorseRacing\horse_racing\utils\prepare_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_after_processing ["where_racecourse"] = df_after_processing ["where_racecourse"].map(pp.extract_place)
C:\Users\endo_\Dropbox\workspace\HorseRacing\horse_racing\utils\prepare_data.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_after_processing["sex"] = df_after_processing["sex_and_age"].map(lambda sex_and_age: sex_and_age[0])
C:\Users\endo_\Dropbox\workspace\HorseRaci

In [11]:
# 学習に使う特徴量のカラム名をリスト化
columns_for_learning = df_for_learning.columns.values.tolist()
columns_for_learning.remove("label_high")
columns_for_learning.remove("label_middle")
columns_for_learning.remove("label_low")

## 学習

In [13]:
# 学習に用いるデータセットの作成
x = np.array(df_for_learning[columns_for_learning])
y = np.array(df_for_learning[["label_high", "label_middle", "label_low"]])
#del df
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=11)
#del x, y
print(x_train.shape)
print(x_test.shape)

(447872, 261)
(191946, 261)


In [14]:
# データセットのシャッフルとバッチ化
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(1024)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1024)

In [15]:
import utils.sample_model as model
    
# モデルのインスタンスを作成
model = model.HorseModel(x_train.shape[1], 3)

In [16]:
metrics = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.AUC(name='auc'),
]
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=metrics)

In [17]:
model.fit(train_ds, epochs=100)

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

438/438 [==============================] - 3s 7ms/step - loss: 1.5730 - accuracy: 0.6734 - auc: 0.6552
Epoch 2/100
438/438 [==============================] - 3s 7ms/step - loss: 0.9921 - accuracy: 0.7005 - auc: 0.7031
Epoch 3/100
438/438 [==============================] - 3s 8ms/step - loss: 0.9700 - accuracy: 0.7053 - auc: 0.7125
Epoch 4/100
438/438 [==============================] - 3s 8ms/step - loss: 0.9538 - accuracy: 0.7098 - auc: 0.7222
Epoch 5/100
438/438 [===========================

438/438 [==============================] - 3s 8ms/step - loss: 0.9159 - accuracy: 0.7224 - auc: 0.7472
Epoch 66/100
438/438 [==============================] - 3s 8ms/step - loss: 0.9150 - accuracy: 0.7225 - auc: 0.7479
Epoch 67/100
438/438 [==============================] - 3s 8ms/step - loss: 0.9151 - accuracy: 0.7222 - auc: 0.7477
Epoch 68/100
438/438 [==============================] - 3s 8ms/step - loss: 0.9149 - accuracy: 0.7224 - auc: 0.7478
Epoch 69/100
438/438 [==============================] - 3s 8ms/step - loss: 0.9146 - accuracy: 0.7223 - auc: 0.7481
Epoch 70/100
438/438 [==============================] - 3s 8ms/step - loss: 0.9144 - accuracy: 0.7227 - auc: 0.7481
Epoch 71/100
438/438 [==============================] - 3s 8ms/step - loss: 0.9143 - accuracy: 0.7226 - auc: 0.7481
Epoch 72/100
438/438 [==============================] - 3s 8ms/step - loss: 0.9140 - accuracy: 0.7226 - auc: 0.7483
Epoch 73/100
438/438 [==============================] - 3s 8ms/step - loss: 0.9139 - 

In [18]:
model.evaluate(test_ds, verbose=2)

188/188 - 1s - loss: 0.9155 - accuracy: 0.7227 - auc: 0.7480


[0.9155256152153015, 0.7227364182472229, 0.747953474521637]

In [ ]:
#テストデータの予測値と正解ラベルの確認
for x, y in zip(x_test, y_test):
    print(f"pred: {model.predict(x.reshape(1, -1))}, label: {y}")

pred: [[0.01628594 0.18641739 0.7972966 ]], label: [0 1 0]
pred: [[0.04779884 0.23023078 0.7219704 ]], label: [0 0 1]
pred: [[0.18886714 0.31825098 0.49288192]], label: [1 0 0]
pred: [[0.17511773 0.40752003 0.41736224]], label: [0 0 1]
pred: [[0.34656137 0.45206794 0.20137069]], label: [0 1 0]
pred: [[0.12507899 0.37721875 0.4977023 ]], label: [0 1 0]
pred: [[0.21506117 0.37362605 0.41131282]], label: [0 0 1]
pred: [[0.40271986 0.37113786 0.22614235]], label: [1 0 0]
pred: [[0.02367876 0.24056304 0.73575824]], label: [0 0 1]
pred: [[0.44908404 0.37145254 0.17946343]], label: [0 1 0]
pred: [[0.33149576 0.3942748  0.2742294 ]], label: [0 1 0]
pred: [[0.5035788  0.33976275 0.15665852]], label: [1 0 0]
pred: [[0.58195984 0.28371373 0.1343264 ]], label: [1 0 0]
pred: [[0.4474886  0.41095358 0.14155781]], label: [0 1 0]
pred: [[0.01285911 0.13553037 0.85161054]], label: [0 1 0]
pred: [[0.234463   0.41764814 0.3478889 ]], label: [0 0 1]
pred: [[0.61354965 0.27196908 0.11448122]], label: [1 0 

pred: [[0.53933114 0.33459592 0.12607288]], label: [0 1 0]
pred: [[0.39773288 0.38383126 0.21843585]], label: [1 0 0]
pred: [[0.47225058 0.31984285 0.20790663]], label: [1 0 0]
pred: [[0.10349572 0.33700418 0.55950016]], label: [0 1 0]
pred: [[0.01700779 0.11030101 0.8726912 ]], label: [0 0 1]
pred: [[0.67180705 0.24192107 0.08627184]], label: [1 0 0]
pred: [[0.5776193  0.31770736 0.10467324]], label: [1 0 0]
pred: [[0.8296909  0.12527002 0.04503909]], label: [1 0 0]
pred: [[0.129753   0.3492951  0.52095187]], label: [0 0 1]
pred: [[0.06297438 0.27007353 0.666952  ]], label: [0 0 1]
pred: [[0.7378745  0.18910953 0.07301599]], label: [0 1 0]
pred: [[0.00293805 0.14599194 0.85107   ]], label: [0 0 1]
pred: [[0.04911063 0.31114787 0.63974154]], label: [0 0 1]
pred: [[0.39897275 0.42445433 0.17657292]], label: [1 0 0]
pred: [[0.09072669 0.33517846 0.57409483]], label: [0 0 1]
pred: [[0.18701418 0.38359216 0.4293937 ]], label: [0 1 0]
pred: [[0.74034595 0.17541005 0.08424403]], label: [0 1 

pred: [[0.03907971 0.2468182  0.714102  ]], label: [0 0 1]
pred: [[0.5287353  0.3484069  0.12285783]], label: [1 0 0]
pred: [[0.19171588 0.32297614 0.48530793]], label: [1 0 0]
pred: [[0.17585634 0.3606727  0.46347088]], label: [1 0 0]
pred: [[0.19387811 0.3732532  0.43286863]], label: [0 1 0]
pred: [[0.19534254 0.4312988  0.37335867]], label: [1 0 0]
pred: [[0.48795977 0.36535075 0.1466894 ]], label: [0 1 0]
pred: [[0.54430354 0.30199814 0.15369827]], label: [0 1 0]
pred: [[0.14732593 0.32011482 0.5325592 ]], label: [0 1 0]
pred: [[0.1450836  0.47386354 0.3810529 ]], label: [0 0 1]
pred: [[0.14186756 0.3686741  0.48945826]], label: [0 0 1]
pred: [[0.11775865 0.34115225 0.5410892 ]], label: [0 1 0]
pred: [[0.2824369  0.41520905 0.30235407]], label: [0 1 0]
pred: [[0.03617142 0.25461265 0.709216  ]], label: [0 0 1]
pred: [[0.46532762 0.35128734 0.18338506]], label: [1 0 0]
pred: [[0.18976864 0.37466365 0.43556774]], label: [0 1 0]
pred: [[0.54246247 0.36210987 0.09542759]], label: [1 0 

pred: [[0.45711544 0.34019747 0.20268701]], label: [1 0 0]
pred: [[0.42425266 0.3306984  0.24504893]], label: [1 0 0]
pred: [[0.27192196 0.33137515 0.39670286]], label: [1 0 0]
pred: [[0.24101402 0.4480096  0.31097636]], label: [0 0 1]
pred: [[0.43669322 0.35403845 0.20926839]], label: [0 1 0]
pred: [[0.49922267 0.32515985 0.17561746]], label: [0 1 0]
pred: [[0.3861811  0.39937317 0.21444573]], label: [0 0 1]
pred: [[0.6810774  0.22694902 0.09197356]], label: [0 1 0]
pred: [[0.6935257  0.22233601 0.08413826]], label: [1 0 0]
pred: [[0.33904827 0.40775508 0.2531967 ]], label: [0 1 0]
pred: [[0.5918592  0.31153852 0.09660228]], label: [1 0 0]
pred: [[0.8579016  0.10367335 0.03842501]], label: [0 0 1]
pred: [[0.0142408  0.14887452 0.8368847 ]], label: [0 0 1]
pred: [[0.06529235 0.31753996 0.6171677 ]], label: [0 1 0]
pred: [[0.6799951  0.22596137 0.09404352]], label: [1 0 0]
pred: [[0.56059873 0.30889845 0.1305028 ]], label: [0 1 0]
pred: [[0.1970127  0.41716793 0.38581935]], label: [0 1 

pred: [[0.3931647  0.37188712 0.23494823]], label: [0 1 0]
pred: [[0.27781564 0.32895723 0.39322716]], label: [1 0 0]
pred: [[0.09470574 0.3978456  0.5074487 ]], label: [0 1 0]
pred: [[0.5111511  0.33319965 0.15564924]], label: [1 0 0]
pred: [[0.16286618 0.4389008  0.39823306]], label: [0 0 1]
pred: [[0.3040096  0.3930711  0.30291927]], label: [1 0 0]
pred: [[0.02781447 0.16547523 0.8067103 ]], label: [0 0 1]
pred: [[0.5581991  0.33164188 0.11015894]], label: [0 1 0]
pred: [[0.5186676  0.35153943 0.129793  ]], label: [0 1 0]
pred: [[0.28890112 0.39989883 0.31120005]], label: [0 1 0]
pred: [[0.0238777  0.25260994 0.7235123 ]], label: [0 1 0]
pred: [[0.12095036 0.28087193 0.59817773]], label: [0 0 1]
pred: [[0.05457034 0.32530093 0.62012875]], label: [0 0 1]
pred: [[0.190036   0.4114554  0.39850864]], label: [0 1 0]
pred: [[0.36395466 0.43507493 0.20097041]], label: [0 1 0]
pred: [[0.04261203 0.2819464  0.6754416 ]], label: [0 0 1]
pred: [[0.28756312 0.3802784  0.33215848]], label: [0 0 

...

pred: [[0.40350136 0.38494036 0.21155816]], label: [1 0 0]
pred: [[0.60971224 0.27127227 0.11901551]], label: [0 0 1]
pred: [[0.18507645 0.39492232 0.42000127]], label: [0 1 0]
pred: [[0.318931   0.37988353 0.3011854 ]], label: [1 0 0]
pred: [[0.02436661 0.24026434 0.7353691 ]], label: [0 0 1]
pred: [[0.31190297 0.38774368 0.30035335]], label: [1 0 0]
pred: [[0.20564014 0.31557003 0.47878987]], label: [1 0 0]
pred: [[0.21959467 0.38971826 0.39068708]], label: [1 0 0]
pred: [[0.18951    0.37327722 0.4372128 ]], label: [0 0 1]
pred: [[0.32741183 0.43705186 0.23553635]], label: [1 0 0]
pred: [[0.21751313 0.36238474 0.42010215]], label: [0 1 0]
pred: [[0.41945875 0.44302434 0.13751693]], label: [0 0 1]
pred: [[0.40781072 0.4527522  0.1394371 ]], label: [0 1 0]
pred: [[0.00497113 0.12959269 0.8654362 ]], label: [0 1 0]
pred: [[0.29696703 0.37531614 0.32771683]], label: [0 1 0]
pred: [[0.15302338 0.45456982 0.3924067 ]], label: [0 1 0]
pred: [[0.2328622  0.366736   0.40040186]], label: [0 1 

pred: [[0.17202024 0.3277534  0.5002264 ]], label: [0 0 1]
pred: [[0.5077035  0.31811774 0.1741789 ]], label: [0 0 1]
pred: [[0.08493154 0.38297728 0.5320912 ]], label: [0 1 0]
pred: [[0.3384485  0.46026698 0.20128447]], label: [1 0 0]
pred: [[0.8470941  0.11125201 0.04165386]], label: [1 0 0]
pred: [[0.8450717  0.12610571 0.0288226 ]], label: [1 0 0]
pred: [[0.5171116  0.31206077 0.17082766]], label: [0 1 0]
pred: [[0.323438   0.3477945  0.32876754]], label: [1 0 0]
pred: [[0.17426671 0.39084098 0.43489236]], label: [1 0 0]
pred: [[0.0730304  0.30948043 0.6174892 ]], label: [0 1 0]
pred: [[0.6304518  0.27671155 0.09283666]], label: [1 0 0]
pred: [[0.00365439 0.0962528  0.90009284]], label: [0 0 1]
pred: [[0.01376384 0.1517517  0.83448446]], label: [0 0 1]
pred: [[0.6444762  0.2805157  0.07500821]], label: [0 0 1]
pred: [[0.3630143  0.4507925  0.18619318]], label: [0 0 1]
pred: [[0.02730717 0.25348583 0.71920705]], label: [0 0 1]
pred: [[0.05353323 0.29653436 0.6499323 ]], label: [0 0 

pred: [[0.40637845 0.38441193 0.20920959]], label: [1 0 0]
pred: [[0.07695955 0.34654662 0.57649386]], label: [0 0 1]
pred: [[0.27796662 0.41499683 0.30703652]], label: [0 1 0]
pred: [[0.2043569  0.36014137 0.4355018 ]], label: [0 1 0]
pred: [[0.6758915  0.24697667 0.07713176]], label: [1 0 0]
pred: [[0.44099894 0.41217396 0.14682703]], label: [0 1 0]
pred: [[0.4205637  0.4297207  0.14971559]], label: [0 1 0]
pred: [[0.10777596 0.42297044 0.4692536 ]], label: [1 0 0]
pred: [[0.2040157  0.44037452 0.3556098 ]], label: [0 1 0]
pred: [[0.655377   0.22954793 0.11507507]], label: [0 1 0]
pred: [[0.4491129  0.36830133 0.18258573]], label: [0 1 0]
pred: [[0.03168928 0.25102025 0.71729046]], label: [0 0 1]
pred: [[0.58119863 0.29710117 0.12170022]], label: [1 0 0]
pred: [[0.29224893 0.44189456 0.26585647]], label: [0 1 0]
pred: [[0.18202765 0.4146012  0.4033711 ]], label: [0 0 1]
pred: [[0.01148195 0.10397863 0.8845395 ]], label: [0 0 1]
pred: [[0.03984207 0.23641936 0.7237386 ]], label: [0 0 

pred: [[0.2548393  0.37259293 0.37256777]], label: [0 1 0]
pred: [[0.07501313 0.36002755 0.56495935]], label: [0 1 0]
pred: [[0.26339152 0.39581    0.34079847]], label: [0 0 1]
pred: [[0.00118646 0.08572868 0.9130848 ]], label: [0 0 1]
pred: [[0.08169693 0.363533   0.55477005]], label: [0 0 1]
pred: [[0.16519015 0.32782143 0.50698847]], label: [0 1 0]
pred: [[0.11112382 0.391161   0.49771523]], label: [0 0 1]
pred: [[0.36515975 0.451125   0.18371524]], label: [0 1 0]
pred: [[0.21457282 0.37974483 0.40568233]], label: [0 1 0]
pred: [[0.41957122 0.41475287 0.16567588]], label: [0 1 0]
pred: [[0.2748888  0.36870158 0.35640964]], label: [0 1 0]
pred: [[0.02745235 0.17510757 0.7974401 ]], label: [0 1 0]
pred: [[0.33611998 0.39674428 0.26713574]], label: [0 0 1]
pred: [[0.15652679 0.39779547 0.44567776]], label: [0 0 1]
pred: [[0.12742284 0.36213708 0.51044005]], label: [0 0 1]
pred: [[0.83961636 0.12427901 0.03610459]], label: [1 0 0]
pred: [[0.35231712 0.40210798 0.24557492]], label: [0 1 

pred: [[0.07556134 0.40115878 0.52327985]], label: [1 0 0]
pred: [[0.75706005 0.17117819 0.07176171]], label: [1 0 0]
pred: [[0.4798221  0.38705295 0.13312496]], label: [0 0 1]
pred: [[0.1168518  0.45008445 0.43306375]], label: [0 0 1]
pred: [[0.12654634 0.29797873 0.5754749 ]], label: [0 0 1]
pred: [[0.4423271  0.36367717 0.19399579]], label: [1 0 0]
pred: [[0.4050621  0.38465586 0.21028197]], label: [1 0 0]
pred: [[0.10390387 0.28700173 0.60909444]], label: [0 0 1]
pred: [[0.02386737 0.20711584 0.76901686]], label: [0 1 0]
pred: [[0.37890103 0.38318557 0.23791344]], label: [1 0 0]
pred: [[0.46445453 0.3779878  0.1575576 ]], label: [1 0 0]
pred: [[0.05457355 0.2267579  0.7186685 ]], label: [0 0 1]
pred: [[0.33557016 0.36866274 0.29576707]], label: [0 0 1]
pred: [[0.0270597  0.24760751 0.7253328 ]], label: [0 0 1]
pred: [[0.2525294  0.32896987 0.41850075]], label: [0 1 0]
pred: [[0.0143603 0.1882682 0.7973715]], label: [0 0 1]
pred: [[0.01936343 0.17800975 0.80262685]], label: [0 0 1]


In [ ]:
# モデルの保存
# io_m.save_model(model, model_name="first_model")

In [ ]:
# 保存したモデルに不具合がないか確認
# model = io_m.read_model("first_model")
# model.evaluate(test_ds, verbose=2)